Приклад коду який контролює доступ до читання/запису полів обʼєкту класу

In [ ]:
class Point:

    def __init__(self, x, y):
        self._x = x
        self.__x = 1
        self._y = y

    @property
    def x(self):
        return 2*self._x

    @x.setter
    def x(self, v):
        if v > 10:
            raise Exception()
        self._x = v

Приклад коду, який реалізує деякий протокол. В даному випадку за протоколом словника
клас має реалізувати метод `__iter__` який повертає пари значень. Ці пари будуть використані
як ключ-значення

In [2]:
class List:
    ...

    def __iter__(self):
        return iter([])

Приклад дескриптора: класа який контролює доступ до аттрибута іншого класу

In [5]:
class TestDescriptor:

    def __set_name__(self, owner, name):
        # set value in the target class by prefixing it with an underscore
        self._name_used = f'_{name}' 
        
        self._object_used = owner
        print(owner, name)

    def __get__(self, instance, owner):
        print(instance, owner)
        # get the value (notice that name used is the prefixed one)
        return getattr(instance, self._name_used)

    def __set__(self, instance, value):
        print(instance, value)
        # set the value (notice that name used is the prefixed one)
        setattr(instance, self._name_used, value)

Приклад дескриптора-валідатора, який обмежує простір можливих значень до значень типу `int`

In [22]:
# create our own error type to make sure it can be deconflicted from other
# errors by caller
class ValidationError(Exception):
    pass


class Int:

    def __init__(self, less_than: int = None, greater_then: int = None):
        self._less_than = less_than
        self._greater_than = greater_then

    def __set_name__(self, owner, name):
        self._name_used = f'_{name}'
        self._object_used = owner

    def __get__(self, instance, owner):
        return getattr(instance, self._name_used)

    def __set__(self, instance, value):
        value_type = type(value)

        if value_type is not int:
            raise TypeError(
                f'Invalid type assigned: '
                f'received `{value_type}`, expected `int`'
            )
        if not (self._less_than < value < self._greater_than):
            raise ValidationError(
                f'Invalid value provided. Expected value in range '
                f'{self._less_than} to {self._greater_than}'
            )
        
        setattr(instance, self._name_used, value)

Приклад використання цього дескриптора в іншому класі. 
Зверніть увагу на читаємість коду, зрозумілість, кількість і в одночас кількість роботи яку він виконує

In [23]:
class User:
    age = Int(less_than=10, greater_then=20)
    
    def __init__(self, age):
        # this is already a descriptor access, not a usual instance write
        self.age = age
    
    def __repr__(self):
        return f'User(age={self.age})'

In [24]:
User(15)

User(age=15)

In [25]:
User(9)

ValidationError: Invalid value provided. Expected value in range 10 to 20

In [26]:
User(25)

ValidationError: Invalid value provided. Expected value in range 10 to 20

Приклад того, як можна використовуючи аннотації коду отримати інформацію про типи, які очікує функція

In [27]:
def plus(a: int, b: int) -> int:
    return a + b


plus.__annotations__

{'a': int, 'b': int, 'return': int}

Приклад застосування: Object-Relational Mapping
[Означення ORM](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping)

In [28]:
import sqlite3

# connection object which allows one to request and receive responses from the database
conn = sqlite3.connect('entertainment.db')


class Field:

    def __set_name__(self, owner, name):
        # query to fetch value
        self.fetch = f'SELECT {name} FROM {owner.table} WHERE {owner.key}=?;'
        # query to store value
        self.store = f'UPDATE {owner.table} SET {name}=? WHERE {owner.key}=?;'

    def __get__(self, obj, objtype=None):
        # on get issue a request to the database to get first value from a certain table
        return conn.execute(self.fetch, [obj.key]).fetchone()[0]

    def __set__(self, obj, value):
        # on set issue a request to the database to set value of a field in the database
        conn.execute(self.store, [value, obj.key])
        conn.commit()


# define some tables
class Movie:
     # Table name
    table = 'Movies'                   
    # Primary key
    key = 'title'
    director = Field()
    year = Field()

    def __init__(self, key):
        self.key = key

class Song:
    table = 'Music'
    key = 'title'
    artist = Field()
    year = Field()
    genre = Field()

    def __init__(self, key):
        self.key = key

Перед тим як можна застосувати ці таблиці треба створити схему в базі даних.
В справжніх ORM це робиться через метакласи та метапрограмування. Наприклад створюється метаклас
який відслідковує які класи створені від нього, генерує DML запит (той що змінює схему а не додає дані).

Задля простоти цього прикладу створимо схему самостійно

In [33]:
conn.execute(
    """CREATE TABLE Movies (
        title varchar(32) primary key,
        director varchar(32),
        year year
    );
    """
)

In [34]:
conn.execute(
    """CREATE TABLE Music (
        title varchar(32) primary key,
        artist varchar(32),
        year year,
        genre varchar(32)
    );
    """
)

In [40]:
conn.execute(
    """insert into Movies values ('Star Wars', 'George Lucas', 1967)"""
)

In [47]:
conn.execute(
    "select * from Movies"
).fetchall()

[('Star Wars', 'George Lucas', 1967)]

In [48]:
Movie('Star Wars').director = 'George Orwell'

In [49]:
conn.execute(
    "select * from Movies"
).fetchall()

[('Star Wars', 'George Orwell', 1967)]

Виставивши значення аттрибута в обʼєкті ми під капотом створили запит в базу даних і поміняли відповідне значеня. 
Очевидно, що приклад далеко не є вичерпним (напр. немає підтримки `primary key`) але ціль цього 
приклада -- проілюструвати підхід

Приклад: імплементувати дескриптор, який буде імітувати поведінку `@property` а саме:
  - дозволить створювати методи які будуть генерувати значення аттрибута "на льоту"
  - дозволить додавати методи для модифікації цих атрибутів
  - дозволить додавати методи для видалення цих атрибутів 

In [51]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        # functions that implement getting, setting and deleting certain attribute
        self.fget = fget
        self.fset = fset
        self.fdel = fdel

        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        self._name = ''

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, objtype=None):
        # if called from class, nothing to generate then return property object itself
        if obj is None:
            return self
        
        # else, if getter is not set raise an error
        if self.fget is None:
            raise AttributeError(
                f'property {self._name!r} of {type(obj).__name__!r} object has no getter'
             )
        # else -- call getter, i.e. call a function that 
        # would generate a value of an attribute
        return self.fget(obj)

    def __set__(self, obj, value):
        # if no setter defined -- raise an error
        if self.fset is None:
            raise AttributeError(
                f'property {self._name!r} of {type(obj).__name__!r} object has no setter'
             )
        # call setter
        self.fset(obj, value)

    def __delete__(self, obj):
        # if no deleter defined -- raise an error
        if self.fdel is None:
            raise AttributeError(
                f'property {self._name!r} of {type(obj).__name__!r} object has no deleter'
             )
        # call deleter
        self.fdel(obj)

    def getter(self, fget):
        """Function to register getter"""
        # create a copy of current instance of class, replace existing **getter** with provided one
        # NOTE: this method can be used as decorator, since it accepts function as a single argument
        prop = type(self)(fget, self.fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def setter(self, fset):
        # create a copy of current instance of class, replace existing **setter** with provided one
        # NOTE: this method can be used as decorator, since it accepts function as a single argument
        prop = type(self)(self.fget, fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def deleter(self, fdel):
        # create a copy of current instance of class, replace existing **deleter** with provided one
        # NOTE: this method can be used as decorator, since it accepts function as a single argument
        prop = type(self)(self.fget, self.fset, fdel, self.__doc__)
        prop._name = self._name
        return prop

In [63]:
class PropertyExample:
    
    def __init__(self, value):
        self._value = value
    
    # NOTE: looking back at the signature of the `__init__` method
    # one can notice that first argument is `fget` which is optional.
    # that means that the next descriptor syntax is equivalent to the following:
    # Property(fset=value)
    @Property
    def value(self):
        print('getter called')
        return 1
    
    @value.setter
    def value(self, value):
        print('setter called')
        self.foo = value

In [59]:
s = PropertyExample(value=1)

In [60]:
s.value

getter called


1

In [61]:
s.value = 2

setter called


In [62]:
s.value

getter called


1

REFERENCES:
- [Python docs, full HOWTO on descriptors](https://docs.python.org/3/howto/descriptor.html)
- [Some random blog post](https://blog.peterlamut.com/2018/11/04/python-attribute-lookup-explained-in-detail/)